In [1]:
import os
from pathlib import Path
import duckdb
import pandas as pd
import numpy as np
from tqdm import tqdm


In [2]:
OUT_DIR = Path("ring_width_output_robust")          
CHUNK_GLOB = str(OUT_DIR / "ring_widths_robust_chunk_*.csv")
RING_PARQUET_DIR = Path("ring_parquet")            
CLIMATE_PARQUET_DIR = Path("climate_parquet")

RING_PARQUET_DIR.mkdir(parents=True, exist_ok=True)
CLIMATE_PARQUET_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
con = duckdb.connect('rings.db')

In [4]:
con.execute(f"""
CREATE OR REPLACE VIEW ring AS
SELECT *
FROM read_csv_auto('{CHUNK_GLOB}', SAMPLE_SIZE=200000, IGNORE_ERRORS=TRUE);
""")

In [6]:
# Peek
print("Rows / year range / columns:")
print(con.execute("""
SELECT COUNT(*) AS n_rows, MIN(year)::INT AS min_year, MAX(year)::INT AS max_year
FROM ring
""").df())

Rows / year range / columns:


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

     n_rows  min_year  max_year
0  16483140       800      2025


In [8]:
print("\nColumns:", con.execute("PRAGMA show_tables;").df())


Columns:    name
0  ring


In [9]:
print(con.execute("DESCRIBE ring").df())

      column_name column_type null   key default extra
0            year      BIGINT  YES  None    None  None
1         tree_id     VARCHAR  YES  None    None  None
2   ring_width_mm      DOUBLE  YES  None    None  None
3       site_code     VARCHAR  YES  None    None  None
4   collection_id     VARCHAR  YES  None    None  None
5       site_name     VARCHAR  YES  None    None  None
6    species_code     VARCHAR  YES  None    None  None
7             lat      DOUBLE  YES  None    None  None
8             lon      DOUBLE  YES  None    None  None
9     elevation_m      DOUBLE  YES  None    None  None
10       province     VARCHAR  YES  None    None  None
11       filename     VARCHAR  YES  None    None  None
12      file_type     VARCHAR  YES  None    None  None


In [10]:
# --- 2) Build site_tree_year (idempotent: averages in case of duplicates) ---
con.execute("""
CREATE OR REPLACE VIEW ring_tree_year AS
SELECT
  site_code,
  tree_id,
  CAST(year AS INT) AS year,
  AVG(ring_width_mm) AS ring_mm
FROM ring
GROUP BY site_code, tree_id, year;
""")

In [12]:
# --- 3) Build site-year chronology (mean, median, n_trees) + z-score safely ---
con.execute("""
CREATE OR REPLACE VIEW chronology_site_year AS
WITH per_site AS (
  SELECT
    site_code,
    CAST(year AS INT) AS year,
    COUNT(DISTINCT tree_id)            AS n_trees,
    AVG(ring_mm)                       AS mean_mm,
    median(ring_mm)                    AS median_mm
  FROM ring_tree_year
  GROUP BY site_code, year
),
site_stats AS (
  SELECT
    site_code,
    AVG(mean_mm)            AS mu,
    stddev_samp(mean_mm)    AS sd
  FROM per_site
  GROUP BY site_code
)
SELECT
  p.*,
  CASE WHEN s.sd > 0 THEN (p.mean_mm - s.mu) / s.sd ELSE NULL END AS mean_mm_z
FROM per_site p
JOIN site_stats s USING (site_code);
""")

In [13]:
print("\nChronology sample:")
print(con.execute("""
SELECT * FROM chronology_site_year
ORDER BY site_code, year
LIMIT 10
""").df())



Chronology sample:


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

  site_code  year  n_trees   mean_mm  median_mm  mean_mm_z
0    can673   800        1  9.880000      9.880   2.408312
1    can673   801        2  8.330000      8.330   1.795734
2    can673   802        3  6.236667      6.980   0.968424
3    can673   803        4  5.978250      6.765   0.866295
4    can673   804        4  5.114000      5.920   0.524733
5    can673   805        6  5.426667      6.615   0.648303
6    can673   806        5  5.764600      7.300   0.781858
7    can673   807        6  5.751667      5.920   0.776746
8    can673   808        5  4.633200      6.050   0.334715
9    can673   809        6  6.190500      7.855   0.950178


In [14]:
# --- 4) (Optional) Materialize ring to partitioned Parquet (province/site_code) ---
# This accelerates repeated queries and downstream joins.
print("\nMaterializing ring → partitioned Parquet (province, site_code) …")
con.execute(f"""
COPY (SELECT * FROM ring)
TO '{RING_PARQUET_DIR.as_posix()}'
(FORMAT 'parquet', COMPRESSION 'ZSTD', PARTITION_BY (province, site_code));
""")
print("Done.")


Materializing ring → partitioned Parquet (province, site_code) …


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Done.
